# Apply on ABDDs demonstration 3

- this demonstrates how tests in analyzing progressive growth of benchmarks during their construction work
- we show here how one clause (from a DIMACS CNF) is added to some accumulated result
- this will keep significantly expanding its size, if canonization after each step is not used

In [ ]:
import itertools
from typing import Optional

from apply.abdd_apply_main import abdd_apply
from apply.abdd import convert_ta_to_abdd
from apply.box_algebra.apply_tables import BooleanOperation
from apply.abdd_node_cache import ABDDNodeCacheClass
from apply.abdd import ABDD, construct_node
from apply.abdd_node import ABDDNode
from apply.evaluation import compare_op_abdd, compare_abdds_tas

from formats.format_vtf import import_treeaut_from_vtf
from formats.render_dot import convert_to_dot
from helpers.utils import box_orders
from helpers.string_manipulation import create_var_order_list

from canonization.unfolding import ubda_unfolding
from canonization.normalization import ubda_normalize
from canonization.folding import ubda_folding
from tree_automata.functions.trimming import remove_useless_states

In [ ]:
op = BooleanOperation.AND

ncache = ABDDNodeCacheClass()
zero = ncache.terminal_0
one = ncache.terminal_1
varcount = 10

n1 = construct_node(8, "X", [zero], "X", [one], ncache)
n2 = construct_node(5, "X", [zero], "X", [n1], ncache)

abdd1 = ABDD('test1', varcount, [n2], rootrule="X")

n4 = construct_node(9, "X", [one], "X", [zero], ncache)
n5 = construct_node(4, "X", [n4], "X", [one], ncache)


abdd2 = ABDD('test2', varcount, [n5], rootrule=None)

convert_to_dot(abdd1)

In [ ]:
convert_to_dot(abdd2)

In [ ]:
abdd_test = abdd_apply(op, abdd1, abdd2, ncache, maxvar=varcount)
ncache.refresh_nodes()
ta = abdd_test.convert_to_treeaut_obj()
ta.reformat_states()
convert_to_dot(abdd_test)

In [ ]:
unf = ubda_unfolding(ta, varcount+1)
convert_to_dot(unf)

In [ ]:
norm = ubda_normalize(unf, create_var_order_list("", varcount+1), fix=True)
norm = remove_useless_states(norm)
norm.reformat_states()
print(compare_abdds_tas(norm, abdd_test))
convert_to_dot(norm)

In [ ]:
fold = ubda_folding(norm, box_orders['full'], varcount+1)
fold.remove_self_loops()
fold = remove_useless_states(fold)
convert_to_dot(fold)

In [ ]:
abdd = convert_ta_to_abdd(fold, ncache=ABDDNodeCacheClass())
print(abdd)
print(compare_abdds_tas(abdd, abdd_test))
convert_to_dot(abdd)


In [ ]:
ncache = ABDDNodeCacheClass()
x = abdd_apply(op, abdd2, abdd, ncache, maxvar=varcount)
convert_to_dot(x)